# Demo of Pydantic
This is a very simple demo of some of the things you can do with Pydantic. 
All we need to do is a `pip install pydantic` and we're good to go! 

We will import BaseModel from pydantic, and then create classes which inherit from BaseModel. Pydantic uses built-in type hinting. 

In [1]:
from pydantic import (
    BaseModel, 
    NegativeInt, 
    PositiveInt, 
    conint, 
    conlist, 
    constr, 
    PositiveFloat
)

In [3]:
# we create a class which inherits from BaseModel. This is a simple class which takes two
# optimal parameters, 'mean' (a float) and 'std' (also a float). 
class Gaussian(BaseModel):
    
    mean: float = 0.0
    std: float = 1.0 

In [4]:
# instantiate the class
data = {"mean": 3.0, "std": 2.0}
gauss = Gaussian(**data)

# Type enforcement

We can print out the class which gives us information about which variables are set

In [5]:
print(gauss)

mean=3.0 std=2.0


Now what if we give the model data with different types? We see that Pydantic automatically converts them to the correct type! The integer passed in is converted to float, and so is the string we pass for std. 

In [6]:
data = {"mean": 4, "std": "5.4"}
gauss= Gaussian(**data)
print(gauss)

mean=4.0 std=5.4


In [7]:
type(gauss.std)

float

Of course if we provide something that cannot easily be cast as a float, we will get an error

In [8]:
data = {"mean": "this is not a float", "std": "5.4"}
gauss= Gaussian(**data)

ValidationError: 1 validation error for Gaussian
mean
  value is not a valid float (type=type_error.float)

# Constrained types

What if we want to make sure that a particular variable does not exceed a certain value? Or what if we want to pass an integer but we want to make sure it's a multiple of 4? We can do this using the constrained variables of Pydantic!

When trying to instantiate the class with values outside of the allowed range, a ValidationError is thrown telling us what was wrong.

In [27]:
class Gaussian(BaseModel):
    
    mean: conint(gt=0, lt=100)  # enforce a mean between 0 and 100
    std: PositiveFloat  # enforce the standard deviation to be positive

    def print_mean(self):
        print(self.mean)

In [23]:
Gaussian(**{"mean": 120, "std": -0.3})

ValidationError: 2 validation errors for Gaussian
mean
  ensure this value is less than 100 (type=value_error.number.not_lt; limit_value=100)
std
  ensure this value is greater than 0 (type=value_error.number.not_gt; limit_value=0)

In [32]:
gauss = Gaussian(**{"mean": "30", "std": 0.3})
gauss.print_mean()

30


# methods

A whole bunch of methods are inherited from BaseModel, e.g. `schema()`, `json()`. For all the methods, just look at the Pydantic docs!

In [11]:
gauss = Gaussian(**{"mean": 4, "std": "5.4"})
gauss.schema()

{'title': 'Gaussian',
 'type': 'object',
 'properties': {'mean': {'title': 'Mean',
   'exclusiveMinimum': 0,
   'exclusiveMaximum': 100,
   'type': 'integer'},
  'std': {'title': 'Std', 'exclusiveMinimum': 0, 'type': 'number'}},
 'required': ['mean', 'std']}

In [12]:
gauss.json()

'{"mean": 4, "std": 5.4}'